## Introduction to Data Science

## The CRISP-DM approach to Data Science Tasks

_________________________

## I. Business Understanding
                    
#### The Business Understanding phase focuses on understanding the objectives and requirements of the project.  
+ Determine business objectives: Determinar formas de auxiliar um estudante internacional recém-formado a conseguir um bom emprego nos EUA   
+ Assess situation:  Muitos dados disponíveis, inclusive com uma mudança política no meio do caminho o que pode gerar resultados interessantes.
+ Determine data mining goals: Coletar dados sobre status das aplicações, empresa, cargo, salário e localização
+ Produce project plan:  Achar padrões e informações interessantes sobre a questão imigratória e nível salarial/de escolaridade. Se possível, no futuro, tentar interagir com outros datasets (como nacionalidade dos imigrantes, grau de escolaridade, e PPP por estado/cidade) e achar informações relevantes para cada emprego, local e salário.

## II. Data Understanding  

+ Collect initial data: Conjunto de dados sobre aprovação de vistos para estudantes internacionais nos EUA entre os anos de 2015 e 2019. Contém informações sobre a classificação do visto, empresa empregadora, área e código do cargo, assim como nome, classificação da jornada de trabalho (tempo integral ou meio-período), salário prevalecente para o cargo dada a empresa, cidade e estado do trabalho. 
+ Describe data:  
+ Explore data:  
+ Verify data quality:  

## III. Data Preparation

+ Select data:   
+ Clean data:   
+ Construct data/transform data/feature engineering:   
+ Integrate data:   
+ (Re) Format data:   

## IV. Modeling

+ Select modeling techniques: 
+ Generate test design: 
+ Build model: 
+ Assess model:  

## V. Evaluation                    

+ Evaluate results: 
+ Review process: 
+ Determine next steps: 

## VI. Deployment

+ Plan deployment: 
+ Plan monitoring and maintenance: 
+ Produce final report: 
+ Review project: 

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=3ac46674-63e8-43c6-afbc-97dcb432b211' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>